# Hunyuan3D-2.1 Test on AeroPath Dataset

This notebook tests the image-to-3D shape generation pipeline using slices from the AeroPath respiratory tract dataset.

## Setup
Install hy3dgen from the Hunyuan3D-2 repo:
```bash
cd models/Hunyuan3D-2
pip install -e .
```

In [1]:
import sys
import os

# Add paths for hy3dgen
sys.path.insert(0, 'models/Hunyuan3D-2')

In [ ]:
import numpy as np
import torch
import trimesh
import matplotlib.pyplot as plt
from IPython.display import Image as IPImage, display

from dataloaders import AeroPathDataset

# Import utilities from organized modules
from utils.preprocessing import prepare_slice_for_hunyuan, normalize_to_unit_range
from utils.visualization import (
    create_mesh_rotation_gif,
    create_comparison_gif,
    plot_slice_with_mask,
    points_to_mesh
)
from utils.metrics import evaluate_reconstruction, print_metrics
from utils.tools import load_mesh, mesh_to_pointcloud


%matplotlib inline

In [3]:
# Uncomment to load the pipeline (requires GPU)
# from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline

## Load AeroPath Dataset

In [4]:
# Load dataset with surface-only points (exterior boundary)
dataset = AeroPathDataset(
    root_dir="data",
    mask_type="lungs",
    plane="coronal",
    surface_only=True  # Extract only surface points for GT
)
print(f"Total samples: {len(dataset)}")

Total samples: 27


In [5]:
# Select a sample index
sample_idx = 8
slice_2d, mask_2d, point_cloud = dataset[sample_idx]

print(f"Slice shape: {slice_2d.shape}")
print(f"Mask shape: {mask_2d.shape}")
print(f"Point cloud shape: {point_cloud.shape} (surface points only)")

Slice shape: torch.Size([1, 487, 602])
Mask shape: torch.Size([1, 487, 602])
Point cloud shape: torch.Size([820077, 3]) (surface points only)


In [6]:
# Visualize the slice and mask using utility function
plot_slice_with_mask(slice_2d, mask_2d, rotate=True)

c:\Users\drago\OneDrive\Desktop\Robitcs Lab\Zero-Shot-SAM3Dfy\Zero-Shot-SAM3Dfy\utils\visualization.py:408: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Prepare Image for Hunyuan3D

In [7]:
# Prepare the image using utility function
input_image = prepare_slice_for_hunyuan(
    slice_2d, mask_2d,
    use_mask_overlay=True,
    rotate=True
)

# Display
plt.figure(figsize=(8, 8))
plt.imshow(input_image)
plt.title('Input Image for Hunyuan3D')
plt.axis('off')
plt.show()

# Save for reference
os.makedirs('outputs', exist_ok=True)
input_image.save('outputs/aeropath_input.png')

C:\Users\drago\AppData\Local\Temp\ipykernel_71344\3946976934.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Load Hunyuan3D Shape Generation Pipeline

Using local weights from `models/Hunyuan3D-2.1`

In [8]:
# Uncomment to load and run the pipeline
# os.environ['HY3DGEN_MODELS'] = 'models'
#
# shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
#     'Hunyuan3D-2.1',
#     subfolder='hunyuan3d-dit-v2-1',
#     use_safetensors=False
# )
#
# # Generate mesh
# mesh = shape_pipeline(image=input_image, num_inference_steps=15)[0]
# mesh.export('outputs/aeropath_mesh.glb')

## Load Existing Mesh and Evaluate

In [9]:
# Load existing mesh using utility function
output_mesh_path = 'outputs/aeropath_mesh.glb'
mesh = load_mesh(output_mesh_path)

print(f"Loaded mesh from: {output_mesh_path}")
print(f"Vertices: {len(mesh.vertices)}, Faces: {len(mesh.faces)}")

Loaded mesh from: outputs/aeropath_mesh.glb
Vertices: 104372, Faces: 208752


In [10]:
# Extract and normalize point clouds
gt_points = point_cloud.numpy()
pred_points = mesh_to_pointcloud(mesh, n_points=200000)

print(f"Ground truth points: {gt_points.shape} (surface only)")
print(f"Predicted points: {pred_points.shape}")

# Normalize to [-1, 1] range using utility functions
gt_points_norm = normalize_to_unit_range(gt_points)
pred_points_norm = normalize_to_unit_range(pred_points)

print(f"\nAfter normalization:")
print(f"GT range: [{gt_points_norm.min():.3f}, {gt_points_norm.max():.3f}]")
print(f"Pred range: [{pred_points_norm.min():.3f}, {pred_points_norm.max():.3f}]")

Ground truth points: (820077, 3) (surface only)
Predicted points: (200000, 3)

After normalization:
GT range: [-1.000, 1.000]
Pred range: [-1.000, 1.000]


## Compute Evaluation Metrics

In [11]:
# Compute all 7 evaluation metrics with ICP alignment
# metrics = evaluate_reconstruction(
#     pc_pred=pred_points_norm,
#     pc_gt=gt_points_norm,
#     threshold=0.01,
#     grid_size=64,
#     normalize=False,  # Already normalized
#     align=True  # Auto-align with ICP
# )

# # Print metrics using utility function
# print_metrics(metrics, title="Reconstruction Quality Metrics (with ICP alignment)")

## Visualization

In [13]:
# Create comparison GIF (GT mesh vs Predicted mesh)
# GT point cloud is converted to mesh via convex hull
sample_name = "9_CT_HR"

comparison_gif_path = create_comparison_gif(
    gt_mesh_or_points=gt_points_norm,  # Will be converted to mesh
    pred_mesh_or_points=mesh,
    output_path='outputs/comparison.gif',
    sample_name=sample_name,
    n_frames=36,
    duration=0.1,
    use_icp_align=False,
    render_mesh=False,  # Render as point clouds with gray shading
    pred_rotation=('y', 90)  # Rotate predicted 90 degrees around x-axis to fix orientation
)

display(IPImage(filename=comparison_gif_path))

KeyboardInterrupt: 